In [1]:
from sklearn.mixture import GaussianMixture
import numpy as np
import joblib
import os

In [ ]:
dir = "/content/drive/MyDrive/Projet/Langues/MFCC/Russian/Preprocessing"


X_train_files_F = [filename for filename in os.listdir(dir + ("/Train/F")) if filename.endswith('txt')]
X_train_files_H = [filename for filename in os.listdir(dir + ("/Train/H")) if filename.endswith('txt')]

X_test_files_F = [filename for filename in os.listdir(dir + ("/Test/F")) if filename.endswith('txt')]
X_test_files_H = [filename for filename in os.listdir(dir + ("/Test/H")) if filename.endswith('txt')]

print(len(X_train_files_F) + len(X_train_files_H))
print(len(X_test_files_F) + len(X_test_files_H))

51
23


In [ ]:
X_train = []
for file_name in X_train_files_F:
    mfcc_array = np.loadtxt(os.path.join(dir + ("/Train/F"), file_name), delimiter=',')
    X_train.append(mfcc_array)
for file_name in X_train_files_H:
    mfcc_array = np.loadtxt(os.path.join(dir + ("/Train/H"), file_name), delimiter=',')
    X_train.append(mfcc_array)

X_train = np.concatenate(X_train, axis=0).reshape(-1,1)

In [ ]:
print(X_train.shape)

(257655, 1)


In [ ]:
X_test = []
for file_name in X_test_files_F:
    mfcc_array = np.loadtxt(os.path.join(dir + ("/Test/F"), file_name), delimiter=',')
    X_test.append(mfcc_array)
for file_name in X_test_files_H:
    mfcc_array = np.loadtxt(os.path.join(dir + ("/Test/H"), file_name), delimiter=',')
    X_test.append(mfcc_array)

X_test = np.concatenate(X_test, axis=0).reshape(-1,1)

In [ ]:
print(X_test.shape)

(119120, 1)


In [ ]:
''' n_components = [16, 32, 64, 128, 256]
gmm_models = [GaussianMixture(n, covariance_type='diag', random_state=42).fit(X_train) for n in n_components] '''

In [4]:
path = "/content/drive/MyDrive/Projet/Langues/GMM/Russian/"

In [ ]:
''' for i in range(len(gmm_models)) :
  joblib.dump(gmm_models[i], path + ("gmm_russian") + str(n_components[i]) + (".gmm")) '''

In [5]:
gmm_loaded = []
for filename in os.listdir(path):
    if filename.endswith('.gmm'):
        model_path = os.path.join(path, filename)
        gmm_model = joblib.load(model_path)
        gmm_loaded.append(gmm_model)

In [ ]:
# Calculate the BIC values for each model
bic_scores = [model.bic(X_test) for model in gmm_loaded]

In [ ]:
bic_scores

[2097699.6724675805,
 2098143.7414775146,
 2099437.862293304,
 2101704.523405053,
 2106306.8009166857]

In [ ]:
best_model_idx = np.argmin(bic_scores)
best_model = gmm_loaded[best_model_idx]

In [ ]:
print(best_model)

GaussianMixture(covariance_type='diag', n_components=16, random_state=42)


In [ ]:
log_likelihoods = [model.score(X_test) for model in gmm_loaded]

In [ ]:
log_likelihoods

[-8.802679406456576,
 -8.802188516807067,
 -8.802910829163192,
 -8.803005636381192,
 -8.803484687050128]

In [ ]:
best_model_idx = np.argmax(log_likelihoods)
best_model = gmm_loaded[best_model_idx]

In [ ]:
print(best_model)

GaussianMixture(covariance_type='diag', n_components=32, random_state=42)


In [8]:
path_test = '/content/drive/MyDrive/Projet/Langues/GMM/Russian/Special Test'

files = os.listdir(path_test)
files = [filename for filename in files if filename.endswith('txt')]
print(files)

scores = []
for filename in files:
  mfcc = np.loadtxt(os.path.join(path_test, filename), delimiter=',').reshape(-1,1)
  score = gmm_loaded[1].score(mfcc)
  scores.append(score)

print(scores)
print(np.max(scores))

['RUSS-5secs.txt', 'RUSS-10secs.txt', 'RUSS-15secs.txt']
[-8.589323925485825, -8.700509164387102, -8.694771469684717]
-8.589323925485825
